In [1]:
from pynq.overlays.base import BaseOverlay
from pynq.lib.video import *
import PIL.Image
import time
import cv2
import numpy as np

base = BaseOverlay("base.bit")
hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

In [2]:
hdmi_in.configure()
hdmi_out.configure(hdmi_in.mode)
#hdmi_in.configure(PIXEL_RGB)
#hdmi_out.configure(hdmi_in.mode, PIXEL_RGB)

hdmi_in.start() #for Switch, set video output to 720p to get PYNQ recognize the stream
hdmi_out.start()

hdmi_in.tie(hdmi_out)

In [3]:
from pynq.lib.arduino import Arduino_IO
import time

arduinoPin0 = None
arduinoPin1 = None
arduinoPin2 = None

def initArduinoPins():
    global base;
    global arduinoPin0;
    global arduinoPin1;
    global arduinoPin2;
    
    arduinoPin0 = Arduino_IO(base.ARDUINO,0,'out')
    arduinoPin0.write(1);
    arduinoPin1 = Arduino_IO(base.ARDUINO,1,'out')
    arduinoPin1.write(1);
    arduinoPin2 = Arduino_IO(base.ARDUINO,2,'out')
    arduinoPin2.write(1);
    return;

def setActiveControlPad():
    global arduinoPin0;
    global arduinoPin1;
    global arduinoPin2;
    arduinoPin0.write(0);
    arduinoPin1.write(0);
    time.sleep(0.05)
    arduinoPin0.write(1);
    arduinoPin1.write(1);
    time.sleep(0.5)
    arduinoPin2.write(0);
    time.sleep(0.05)
    arduinoPin2.write(1);
    return;

def pressJumpOnce():
    global arduinoPin0;
    global arduinoPin1;
    global arduinoPin2;
    arduinoPin2.write(0);
    time.sleep(0.05)
    arduinoPin2.write(1);
    return;
    

initArduinoPins();

In [16]:
pressJumpOnce()

In [14]:
setActiveControlPad()

In [ ]:


scoreTLCorner = (1065,516)
scoreBRCorner = (1214,575)

prim = None
firstFrame = True
start = time.time()
currentTime = time.time()
runTime = 600

needToJmp = False
jmpTime = 0


lastRopeHitGroundTime = 0;

while ((currentTime-start)<runTime):
    scoreRectColor = (0,0,255)
    #use button 0 to exit
    if base.buttons[0].read():
        break;
    frame = hdmi_in.readframe()
    scoreAreaColor = frame[scoreTLCorner[1]:scoreBRCorner[1], scoreTLCorner[0]:scoreBRCorner[0]]
    scoreArea = cv2.cvtColor(scoreAreaColor, cv2.COLOR_BGR2GRAY)
    ret, scoreArea = cv2.threshold(scoreArea, 200, 255, cv2.THRESH_BINARY)
    img = PIL.Image.fromarray(scoreArea)
    
    
    if (time.time()>=jmpTime+0.1):
        arduinoPin2.write(1);
    
    if needToJmp and (time.time()>=jmpTime):
        needToJmp = False;
        arduinoPin2.write(0);
        print(currentTime,"JMP")
    
    
    if not firstFrame:
        
        diffo = ((np.sum(np.abs(np.subtract(scoreArea, prim))))/1000)
        #print(diffo)
        if (diffo>10):
            cornerColor = scoreAreaColor[0,0]
            if (cornerColor[0]>(16-5)) and (cornerColor[0]<(16+5)):
            
                #16 153 176
                ropeHitGroundTime = time.time();
                ropeInterval = ropeHitGroundTime-lastRopeHitGroundTime;
                
                if (ropeInterval<1.5):
                    needToJmp = True; 
                    jmpTime = ropeHitGroundTime + ropeInterval - 0.3;
                    print("next JMP",jmpTime," ITV",ropeInterval)
                    scoreRectColor = (0,255,0)
                
                lastRopeHitGroundTime = ropeHitGroundTime
                

                print(time.time(),cornerColor)
                #arduinoPin2.write(0);
    
    prim = scoreArea.copy()
    firstFrame = False
    #frame = cv2.rectangle(frame, scoreTLCorner, scoreBRCorner, scoreRectColor)
    #hdmi_out.writeframe(frame)
    currentTime = time.time()
    


hdmi_in.tie(hdmi_out)


1587182543.547365 [ 16 153 176]
next JMP 1587182543.2808576  ITV 0.016754627227783203
1587182543.565089 [ 16 153 176]
1587182543.5677726 JMP
1587182548.2808661 [ 16 153 176]
next JMP 1587182550.413991  ITV 1.2165703773498535
1587182549.4992945 [ 16 153 176]
1587182550.4134665 JMP
next JMP 1587182551.5643587  ITV 1.1834690570831299
1587182550.6819718 [ 16 153 176]
1587182551.5634649 JMP
next JMP 1587182552.714471  ITV 1.1667907238006592
1587182551.8487527 [ 16 153 176]
1587182552.7134197 JMP
next JMP 1587182553.9140453  ITV 1.1831824779510498
1587182553.0327237 [ 16 153 176]
1587182553.913405 JMP
next JMP 1587182554.864044  ITV 1.0665905475616455
1587182554.09845 [ 16 153 176]
1587182554.8634357 JMP
next JMP 1587182555.8975215  ITV 1.0500340461730957
1587182555.1493628 [ 16 153 176]
1587182555.8967657 JMP
next JMP 1587182556.9808006  ITV 1.0666565895080566
1587182556.216016 [ 16 153 176]
1587182556.9801238 JMP
next JMP 1587182558.0483046  ITV 1.0670802593231201
1587182557.2823033 [ 16 1

In [ ]:
hdmi_out.close()
hdmi_in.close()